#### 지도학습기반 감성분석 실습 - IMDB 영화 리뷰 데이터

In [5]:
# 데이터셋 로드
import pandas as pd
review_df=pd.read_csv('./labeledTrainData.tsv',header=0,sep='\t',quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [6]:
# review 하나 출력
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [7]:
import re

# <bn> 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

In [8]:
# 트레인 셋, 테스트셋 나누기
from sklearn.model_selection import train_test_split

class_df=review_df['sentiment']
feature_df=review_df.drop(['id','sentiment'],axis=1,inplace=False)

X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [9]:
# 필요한 라이브러리 임포트
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import  Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [10]:
# 텍스트 테이터를 수치적 특성 데이터셋으로 변환하는 카운트 벡터화와 
# 분류를 수행하는 로지스틱 회귀 모델을 파이프라인으로 구성
Pipeline = Pipeline([
    ('cnt_vect',CountVectorizer(stop_words='english',ngram_range=(1,2))),
    ('lr_clf',LogisticRegression(solver='liblinear',C=10))])

In [11]:
Pipeline.fit(X_train['review'],y_train)
pred=Pipeline.predict(X_test['review'])


In [14]:
# ROC-AUC  점수 계산을 위해 예측 확률을 구합니다.
pred_probs=Pipeline.predict_proba(X_test['review'])[:,1]

# 정확도와 ROC-AUC 점수를 출력합니다
print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test,pred),
                               roc_auc_score(y_test,pred_probs)))

예측 정확도는 0.8867, ROC-AUC는 0.9501


### 비지도학습 기반 감성 분석 소개
### SentiWordNet을 이용한 Sentiment Analysis 
* WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해

#### 영어에서 present는 선물도 있지만 현재란 의미도 있음
#### 우리나라에서 밥 먹었어?는 실제 밥을 먹었는가를 묻는 의미도 있지만 인사말로 쓰이기도함
#### 이렇게 타겟이 정해지지 않은 상태에서의 감성 분석은 매우 어려운 분야임
#### 따라서, 다양한 상황에서 같은 어휘라도 다르게 사용되는 시맨틱 (문맥상 의미)정보를 제공받아 각각의 품사(명사, 동사, 형용사 등)로 구성된 개별단어를 사용하여 표현하여야 함
#### wordnet도 이러한 시맨틱을 제공하지만 성능이 좋지 않으므로 sentiwordnet과  sentiment analysis 라이브러리를 사용함

In [15]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipp

True

In [16]:
# present를 synset에 태우기
from nltk.corpus import wordnet as wn
term = 'present'

# present라는 단어로 wordnet의 synsets 생성
synsets=wn.synsets(term)

print('synsets() 반환 type :',type(synsets))
print('synsets() 반환 값 개수 :',len(synsets))
print('synsets() 반환 값 :', synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 개수 : 18
synsets() 반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [17]:
# 각각의 의미를 출력
for synset in synsets :
    print('##### Synset name : ', synset.name(),'#####')
    print('POS :',synset.lexname())  #품사명
    print('Definition:',synset.definition())  #정의
    print('Lemmas:',synset.lemma_names())  #부명제
    print('')

##### Synset name :  present.n.01 #####
POS : noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']

##### Synset name :  present.n.02 #####
POS : noun.possession
Definition: something presented as a gift
Lemmas: ['present']

##### Synset name :  present.n.03 #####
POS : noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']

##### Synset name :  show.v.01 #####
POS : verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']

##### Synset name :  present.v.02 #####
POS : verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']

##### Synset name :  stage.v.01 #####
POS : verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'rep

In [18]:
# synset 객체 생성
tree=wn.synset('tree.n.01')
lion=wn.synset('lion.n.01')
tiger=wn.synset('tiger.n.02')
cat=wn.synset('cat.n.01')
dog=wn.synset('dog.n.01')

entities=[tree,lion,tiger,cat,dog]
similarities=[]
entity_names=[entity.name().split('.')[0] for entity in entities]

In [19]:
# 단어별 synset 들을 iteration 하면서 다른 단어들의 synset과 유사도를 측정합니다. 
for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity), 2)  for compared_entity in entities ]
    similarities.append(similarity)
    
# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame형태로 저장합니다.  
similarity_df = pd.DataFrame(similarities , columns=entity_names,index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [21]:
# nltk에서 재배포한 sentiwordnet 임포트
from nltk.corpus import sentiwordnet as swn

# 'slow' 라는 단어에 대한 모든 감정 시맨틱을 조회
senti_synsets=list(swn.senti_synsets('slow'))

In [22]:
# 조회된 감정 synset들의 데이터 타입을 출력합니다.
print('senti_synsets() 반환 type :', type(senti_synsets))

# 조회된 감정 synset의 개수를 출력합니다.
print('senti_synsets() 반환 값 갯수:', len(senti_synsets))

# 조회된 감정 synset의 실제 리스트를 출력합니다.
print('senti_synsets() 반환 값 :', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 갯수: 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [24]:
## 객관성과 감성을 지수화

import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())           

print('\n')

fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ',fabulous.pos_score())
print('fabulous 부정감성 지수: ',fabulous.neg_score())


father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125


### 감상평 감성 분석 플로우
#### 1. 문서를 문장 단위로 분해
#### 2. 문장을 단어 단위로 토큰화 하고 품사 태깅
#### 3. 품사 태깅된 단어 기반으로 synset 객체와 senti_synset객체를 생성
#### 4. Senti_synset 객체에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산하여 특정값 이상일때 긍정으로, 아닐때 부정으로 분류

In [26]:
# 간단한 품사 태그 함수

def penn_to_wn(tag):
    if tag.startswith('J'):  #형용사
        return wn.ADJ
    elif tag.startswith('N'):  #명사
        return wn.NOUN
    elif tag.startswith('R'):  #부사
        return wn.ADV
    elif tag.startswith('V'):  #동사
        return wn.VERB
    return 

In [27]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    # 감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0
    
    # WordNetLemmatizer 객체 생성
    lemmatizer = WordNetLemmatizer()
    # 입력된 텍스트를 문장으로 분해
    raw_sentences = sent_tokenize(text)
    
    # 분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        # NTLK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        #강아지가 매우 빨리 뜁니다에서 [(강아지, n), (매우, r), (빨리, r), (뜁니다, v)]
        for word, tag in tagged_sentence:
            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            
            # 명사, 형용사, 동사에 해당하지 않는 품사는 스킵
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체 생성
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            
            if not lemma:
                continue
            
            # 생성한 어근과 품사 태깅으로부터 Synset 객체 추출
            synsets = wn.synsets(lemma, pos=wn_tag)
            
            if not synsets:
                continue
            
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            
            # 긍정 감성 지수는 +로, 부정 감성 지수는 -로 합산하여 감성 지수 계산
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count += 1
    
    # 분석된 단어 토큰이 없으면 감성 지수 0 반환
    if not tokens_count:
        return 0
    
    # 총 score가 0 이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0:
        return 1
    
    return 0



In [28]:
review_df['preds']=review_df['review'].apply(lambda x: swn_polarity(x))
y_target=review_df['sentiment'].values
preds=review_df['preds'].values

In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score
import numpy as np

print(confusion_matrix( y_target, preds))
print("정확도:", np.round(accuracy_score(y_target , preds), 4))
print("정밀도:", np.round(precision_score(y_target , preds),4))
print("재현율:", np.round(recall_score(y_target, preds), 4))
print("F1 스코어:", np.round(f1_score(y_target, preds), 4))

[[7650 4850]
 [3549 8951]]
정확도: 0.664
정밀도: 0.6486
재현율: 0.7161
F1 스코어: 0.6807
